#***Bix Tecnologia Challenge***

##Candidato avaliado
* Helder da Silva Galbier

##Situação Proposta

* Custos de manutenção reduzidos para sistemas de ar em uma frota de caminhões.
* Listar fatores que indicam falha no sistema de manutenção utilizado atualmente.



In [ ]:
# preparing drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##[Análise exploratória de dados e Dataprep]

**1. Que passos você seguiria para resolver esse problema? Por favor descreva da forma mais completa e clara possível todas as etapas que você considera essenciais para resolver o problema.**

Alguns pontos importantes a serem considerados:
* A empresa quer reduzir custos de manutenção apenas para caminhões com problemas no sistema de ar. Portanto, caminhões com outros tipos de problemas não podem ser incluídos no modelo ML.
* As colunas do conjunto de dados foram codificadas. Isto pode comprometer a precisão da análise do modelo de Machine Learning aplicado. Mesmo assim, será feito um estudo.
* Presumi que cada coluna do conjunto de dados corresponde a uma ocorrência de manutenção para cada veículo.

Portanto, optei por implementar a preparação dos dados da seguinte forma: considere apenas o número total de caminhões com problemas no sistema de ar. Primeiro, vamos contar quantos caminhões estão em manutenção neste ano e nos últimos anos.



In [ ]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# importing files

present_file_path = '/content/drive/MyDrive/bix_tech/air_system_present_year.csv'
previous_file_path = '/content/drive/MyDrive/bix_tech/air_system_previous_years.csv'

df_present = pd.read_csv('/content/drive/MyDrive/bix_tech/air_system_present_year.csv')
df_previous = pd.read_csv('/content/drive/MyDrive/bix_tech/air_system_previous_years.csv')

# counting trucks with air system maintenance this year

count_pres = df_present['class'].value_counts().get('pos', 0)
print(f"There are {count_pres} trucks in maintenance this year")

# counting trucks with another issues this year

count_pres_other = df_present['class'].value_counts().get('neg', 0)
total_pres = count_pres + count_pres_other
print(f"There are {count_pres_other} trucks in maintenance this year for another issues")
print(f"There are {total_pres} trucks in this dataset")
print("----------------------------------------------------------------------------------------------------------------------")

# counting trucks with air system maintenance in recent years

count_prev = df_previous['class'].value_counts().get('pos', 0)
print(f"There are {count_prev} trucks in maintenance in recent years")

# counting trucks with another issues in recent years

count_prev_other = df_previous['class'].value_counts().get('neg', 0)
total_prev = count_prev + count_prev_other
print(f"There are {count_prev_other} trucks in maintenance in recent years for another issues")
print(f"There are {total_prev} trucks in this dataset")
print("----------------------------------------------------------------------------------------------------------------------")

# [dataprep] Removing rows that contain "na" (current year) and filtering the rows thar contains "pos"

df_present.replace('na', pd.NA, inplace=True)
df_cleaned = df_present.dropna()
df_cleaned_pos_pres = df_cleaned[df_cleaned['class'] == 'pos']
print(df_cleaned_pos_pres.head())
print("----------------------------------------------------------------------------------------------------------------------")

# [dataprep] Removing rows that contain "na" (past years) and filtering the rows thar contains "pos"

df_previous.replace('na', pd.NA, inplace=True)
df_cleaned = df_previous.dropna()
df_cleaned_pos_prev = df_cleaned[df_cleaned['class'] == 'pos']
print(df_cleaned_pos_prev.head())

There are 375 trucks in maintenance this year
There are 15625 trucks in maintenance this year for another issues
There are 16000 trucks in this dataset
----------------------------------------------------------------------------------------------------------------------
There are 1000 trucks in maintenance in recent years
There are 59000 trucks in maintenance in recent years for another issues
There are 60000 trucks in this dataset
----------------------------------------------------------------------------------------------------------------------
     class   aa_000 ab_000 ac_000 ad_000 ae_000 af_000 ag_000  ag_001  \
486    pos  1172556      0    246    326      0      0      0       0   
583    pos   354598      0   4944   4648      0      0      0       0   
728    pos   468666      2   2596   1794      0      0      0  194100   
1348   pos   251818      0   3082   1238      0      0     30    3668   
1687   pos   195268      0    472    380      0      0      0       0   

      

##[Escolha do modelo]

**2. Qual métrica técnica de ciência de dados você usaria para resolver esse desafio? Ex: erro absoluto, rmse, etc.**

Após separar os dados entre o número absoluto de caminhões com problemas no sistema de ar e o número absoluto de caminhões com outros problemas de manutenção, tanto no conjunto de dados com informações de anos anteriores quanto no conjunto de dados do ano mais atual, tornou-se mais sugestivo implementar um modelo de Regressão Logística para cada conjunto de dados, considerando o número absoluto de caminhões com problemas no sistema aéreo sobre o número total de caminhões. Esta abordagem prevê o valor de dados desconhecidos usando outro valor de dados relacionado e conhecido.
Para este modelo, utilizei a métrica de acurácia que define a precisão do modelo de regressão utilizado.
Considerei 20% como parâmetro de proporção de dados para o teste.

In [ ]:
  # loading regression model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# splitting data between "test" and "training" for df_cleaned_pos_pres

x_train, x_test, y_train, y_test = train_test_split(df_cleaned_pos_pres['ab_000'],df_cleaned_pos_pres['ee_009'],test_size=0.2)

# splitting data between "test" and "training" for df_cleaned_pos_pres

x_train, x_test, y_train, y_test = train_test_split(df_cleaned_pos_prev['ab_000'],df_cleaned_pos_prev['ee_009'],test_size=0.2)

# creating Logistics Regression model

model_df_pres = LogisticRegression()
model_df_prev = LogisticRegression()

Tendo criado os modelos de regressão, agora vamos treiná-los e inserir alguns valores de previsão.

In [ ]:
# training model

model_df_pres.fit(np.array([x_train]).reshape(-1, 1),y_train)
model_df_prev.fit(np.array([x_train]).reshape(-1, 1),y_train)

# inputing prediction values

model_df_pres.predict(np.array([800]).reshape(-1,1))
model_df_prev.predict(np.array([90]).reshape(-1,1))

# checking the accuracy of the implemented model

accuracy_pres = model_df_pres.score(np.array([x_test]).reshape(-1, 1),y_test)
accuracy_prev = model_df_prev.score(np.array([x_test]).reshape(-1, 1),y_test)
print(f"The accuracy of this model for current data is {accuracy_pres}")
print(f"The accuracy of this model for data from previous years is {accuracy_prev}")

The accuracy of this model for current data is 0.875
The accuracy of this model for data from previous years is 0.875


**3. Qual métrica de negócios você usaria para resolver o desafio?**


Como métricas de negócio para apoiar decisões na resolução deste problema, considero:
- Tempo médio entre falhas: indicador que mostra o tempo médio de utilização de um veículo antes de ocorrer uma falha. Quanto maior o tempo indicado, maior confiabilidade na frota de caminhões;
- Índice de falhas: Indicador que indica o percentual de caminhões que necessitam de manutenção corretiva em relação ao número absoluto de veículos;
- Custo de manutenção preventiva versus corretiva: considero uma boa métrica de negócio, pois podemos comparar custos com manutenção corretiva e preventiva. O aumento dos custos corretivos pode significar maior necessidade de manutenção preventiva (a mais importante);
- Tempo médio de reparo: indicador que mede o tempo necessário para concluir a manutenção de um veículo. É uma métrica que também verifica a eficiência da manutenção;

**4. Como as métricas técnicas se relacionam com as métricas de negócios?**

Conceitualmente, a regressão logística trata de um modelo estatístico usado para prever a probabilidade de ocorrência de algum evento. Em outras palavras, possui dois resultados possíveis, como “sim/não”, “sucesso/falha”, “pos/neg”, etc.
O modelo apresentado demonstrou uma precisão de 87,5%, considerada relativamente alta. A aplicabilidade deste modelo às métricas de negócios discutidas anteriormente são:
- Tempo Médio entre Falhas: o modelo de regressão logística pode ser utilizado para prever a probabilidade de um caminhão falhar dentro de um determinado período de tempo. Ao explorar os fatores que induzem falhas, podemos identificar os principais preditores que aumentam ou diminuem essa métrica com maior precisão/exatidão. Com base nesta previsão, a empresa pode estudar estratégias de manutenção preventiva;
- Taxa de falhas: O modelo de regressão logística pode prever a probabilidade de falha de cada caminhão. A soma dessas probabilidades indica a taxa de falha de toda a frota. Dessa forma, a empresa pode estudar uma melhor alocação de recursos e desenvolver um melhor planejamento de manutenção preventiva, visando reduzir o índice de falhas;
- Custo de manutenção preventiva versus corretiva: com o modelo de regressão logística podemos determinar quais caminhões correm maior risco de necessitar de manutenção corretiva. Isto permite planejar a manutenção preventiva com mais eficiência, reduzindo os custos totais de manutenção. A análise de custo e benefício pode ser melhorada, mostrando que um maior investimento em manutenção preventiva pode reduzir significativamente os custos corretivos;
-Tempo médio de reparo: o modelo de regressão ajuda a prever a probabilidade de diferentes tipos de falhas, considerando o tempo de reparo para cada evento. Assim, a empresa consegue otimizar os recursos operacionais utilizados na cadeia logística de manutenção de veículos;

**5. Que tipos de análises você gostaria de realizar no banco de dados de clientes**

O comunicado refere que “por questões burocráticas relacionadas com os contratos da empresa, todas as colunas tiveram que ser codificadas”. Esta codificação de dados pode distorcer os resultados do modelo de Machine Learning implementado. Para uma análise eficaz, é essencial trabalhar numa base de dados com informações tão precisas quanto possível. Mais análises poderiam ser implementadas, tais como:
- Análise exploratória de dados;
- traçar medidas de tendência central;
- traçar medidas de dispersão;
- rastrear ofensores (locais, rotas, tipo de veículos, incidência por condutor, etc.);

Tudo isso, claro, respeitando as diretrizes de Governança Corporativa, leis vigentes de proteção de dados, entre outras restrições legais e administrativas.

**6. Quais técnicas você usaria para reduzir a dimensionalidade do problema**

Uma importante técnica de ciência de dados que eu usaria é o Clustering, como o método K-Means, amplamente utilizado para verificar padrões em conjuntos de dados e agrupá-los por características semelhantes.
Outra técnica que eu usaria é a seleção de características (Feature Selection). Além do método já proposto (Baseado em Modelo), métodos estatísticos como análise de variância e testes de correlação ajudariam a identificar quais características deste conjunto de dados estão mais intimamente relacionadas aos custos de manutenção.


**7. Que técnicas você usaria para selecionar variáveis ​​para seu modelo preditivo?**

Além da entrada de dados, que preenche os valores de NA com algum valor razoável (geralmente alguma medida de tendência central), eu também usaria:
- Escalonamento de variável: transformar dados em uma escala comum, a fim de reduzir a dominância de grandes magnitudes de dados sobre magnitudes menores, juntamente com a normalização Min-Max;
- Codificação de variáveis ​​categóricas: com as técnicas de codificação one-hot (uma coluna para cada categoria) e codificação ordinal (mapeamento de categorias com valores em números);


**8. Quais modelos preditivos você usaria ou testaria para esse problema? Por favor indique pelo menos 3.**

Além do modelo de Regressão Logística utilizado, eu também utilizaria:
- Regressão Linear (método supervisionado), que prevê valores contínuos e são exatamente aqueles utilizados no conjunto de dados;
- K-Means (não supervisionado): é um método de agrupamento que divide os dados em conjuntos semelhantes. Ajuda a identificar padrões no conjunto de dados;
- K-vizinhos mais próximos (supervisionado): classifica cada amostra do conjunto de dados considerando a distância em relação aos seus vizinhos. Identificaria, por exemplo, quais rotas causam mais danos aos caminhões.

**9. Como você avaliaria qual dos modelos treinados é o melhor?**


Penso que o modelo de Clustering K-means seria o mais adequado para uma primeira fase de análise, pois ajudaria a identificar padrões semelhantes no conjunto de dados. Isso facilitaria a identificação de:
- quais rotas apresentam maior incidência de veículos sinistrados;
- quais motoristas possuem mais registros de manutenção;
- quanto cada rota ou motorista contribui para o custo de manutenção;
- frequência de veículos que necessitam de manutenção, etc.;

**10. Como você explicaria o resultado do seu modelo? É possível saber quais variáveis ​​são mais importantes?**

Inicialmente, o modelo de Regressão Logística apresentou acurácia de 87,5%. É considerada uma alta taxa de precisão para um modelo de Machine Learning aplicado a problemas de negócio. Vale a pena apresentar aqui gráficos para visualizar melhor essa precisão e os impactos que esse modelo terá nos custos de manutenção. Os coeficientes do modelo, num modelo de regressão, são os maiores influenciadores nas variáveis ​​de resposta, dependendo da sua distância da média.



**11. Como você avaliaria o impacto financeiro do modelo proposto?**

Com 87,5% de precisão, o modelo é eficaz na previsão de possíveis falhas. Isso significa que, em média, o modelo faz previsões corretas em 87,5% das vezes.
Em relação à redução de falhas: é possível estimar o percentual de falhas que podem ser evitadas com intervenções preventivas baseadas em previsões de modelos. Dessa forma, estima-se uma economia nos custos de manutenção, pois a empresa poderá ter uma visão geral de onde aloca recursos para reduzir seus custos de manutenção.

**12. Quais técnicas você usaria para realizar a otimização de hiperparâmetros do modelo escolhido?**

A otimização de hiperparâmetros engloba a busca pelos melhores valores de parâmetros que não são aprendidos diretamente pelo modelo, mas que influenciam seu desempenho. Algumas técnicas que eu aplicaria a este modelo:
- Grid search: garantindo a avaliação de todas as combinações possíveis num espaço definido;
- Otimização Bayesiana: dentro de um mesmo espectro de análise, tenderá a encontrar os melhores hiperparâmetros com menos avaliações, em comparação com a busca em grade ou aleatória;

**13. Que riscos ou precauções você apresentaria ao cliente antes de colocar este modelo em produção?**

Em qualquer aplicação de um modelo de Machine Learning, o primeiro cuidado é com a qualidade dos dados (dados ausentes, outliers, escala variável). Este é um ponto importante a ser reforçado com o cliente porque um modelo de Machine Learning alimentado e treinado em um conjunto de dados tendencioso trará resultados ineficientes ou inconclusivos.

**14. Se o seu modelo preditivo for aprovado, como você o colocaria em produção?**

Após processar e preparar os dados, treinar e validar o modelo, precisamos seguir alguns passos para colocá-lo em produção:
- Serialização do modelo, que consiste em salvar e carregar o modelo a ser utilizado em produção. Em Python existem bibliotecas específicas para isso como "pickle" ou "joblib";
- Versionar o modelo em um sistema de versionamento como o Git;
- Criar uma API para o modelo, configurando um ambiente onde o modelo será disponibilizado;

**15. Se o modelo estiver em produção, como você o monitoraria?**

Para monitorar qualquer sistema precisamos observar alguns detalhes essenciais, como:
- monitoramento de desempenho, observando latência de previsão, taxa de erro e outras métricas. Algumas ferramentas podem ajudar nessa tarefa, como o Grafana;
- Implementar logging para rastrear previsões e erros;



**16. Se o modelo estiver em produção, como você saberia quando treiná-lo novamente?**

- Garantir que o modelo esteja atualizado, treinando-o periodicamente com novos dados. No caso específico da transportadora, mensalmente é uma frequência razoável.